In [1]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
assistant_id = "asst_5N02i9PqB3VVqUYSAbkm76WH"

assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)

Assistant(id='asst_5N02i9PqB3VVqUYSAbkm76WH', created_at=1711067256, description=None, file_ids=[], instructions='Your name is Mara. You are a helpful AI assistant that provides information about a company such as earnings call transcripts, stock information such as current price, 52 week high and low price, and company information such as industry, summary. You get all your information from the functions available to you only. You only answer questions based on the information you have been given. You have access to a sandboxed environment for writing and testing code. If need clarifications, please ask the user to provide more information. You are given various functions that provide all the data you need to answer questions. You might have to call multiple functions to get all needed information.', metadata={}, model='gpt-35-turbo', name='Company-Insights', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter'), ToolFunction(function=FunctionDefinition(name='send_su

In [3]:
thread = client.beta.threads.create()
print(thread.id)

thread_V7p5t3ce9JDe48iXQpHs5y44


In [14]:
# Add a user question to the thread
msg = "any news for JNJ"

msg = "email me a summary of this conversation to ddiaz@live.com thread_id: thread_V7p5t3ce9JDe48iXQpHs5y44"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=msg
)

thread_messages = client.beta.threads.messages.list(thread.id)

In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [16]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

requires_action


In [7]:
from functions.email import send_summary

#send_summary("ddiaz@live.com", thread.id)

In [17]:
from functions.company import get_company_info
from functions.news import get_news
from functions.stock import get_stock_info
from functions.email import send_summary

import time

max_steps = 10 
sleep = .5

#add the available functions here   
available_functions = {"get_company_info": get_company_info, "get_news": get_news, "get_stock_info": get_stock_info, "send_summary": send_summary}


try:
    cnt = 0
    while cnt < max_steps:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)        
        cnt += 1
        if run.status == "requires_action":
            tool_responses = []
            if (
                run.required_action.type == "submit_tool_outputs"
                and run.required_action.submit_tool_outputs.tool_calls is not None
            ):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls

                for call in tool_calls:
                    if call.type == "function":
                        if call.function.name not in available_functions:
                            raise Exception("Function requested by the model does not exist")
                        function_to_call = available_functions[call.function.name]
                        print(call.function.arguments)
                        print(call.function.name)
                        tool_response = function_to_call(**json.loads(call.function.arguments))
                        tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

            print(run.status)

        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "completed":
            break
        time.sleep(sleep)

except Exception as e:
    print(e)



{
  "email_address": "ddiaz@live.com",
  "thread_id": "thread_V7p5t3ce9JDe48iXQpHs5y44"
}
send_summary
Number of messages: 5
B4: any news for JNJ
ATR: any news for JNJ
B4: Here are some recent news articles about Johnson & Johnson (JNJ):

1. "Johnson & Johnson partners with NVIDIA to accelerate AI adoption in surgery" - Johnson & Johnson has partnered with chip manufacturer NVIDIA to leverage artificial intelligence in surgical applications. The collaboration aims to enable real-time analysis and global availability of AI algorithms for surgical decision-making. [Read more](source1)

2. "Johnson & Johnson embraces AI for healthcare innovation" - Johnson & Johnson recognizes the potential of AI to revolutionize healthcare and is committed to implementing AI technology to drive innovation in the sector. The company is harnessing advanced technologies such as AI, machine learning, data science, and digital health as part of its digital transformation strategy. [Read more](source2)

3. "Jo

In [9]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
print(run.status)

{
  "data": [
    {
      "id": "msg_yqKb9C90qwHO33u46lhv3mhO",
      "assistant_id": "asst_5N02i9PqB3VVqUYSAbkm76WH",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Here are some recent news articles about Johnson & Johnson (JNJ):\n\n1. \"Johnson & Johnson partners with NVIDIA to accelerate AI adoption in surgery\" - Johnson & Johnson has partnered with chip manufacturer NVIDIA to leverage artificial intelligence in surgical applications. The collaboration aims to enable real-time analysis and global availability of AI algorithms for surgical decision-making. [Read more](source1)\n\n2. \"Johnson & Johnson embraces AI for healthcare innovation\" - Johnson & Johnson recognizes the potential of AI to revolutionize healthcare and is committed to implementing AI technology to drive innovation in the sector. The company is harnessing advanced technologies such as AI, machine learning, data science, and digital health as part of its digi